In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import joblib
import sklearn
# Import Scikit-Learn library for decision tree models
import sklearn 

from sklearn import datasets, linear_model, metrics, tree
from sklearn.utils import resample

from sklearn.tree import DecisionTreeRegressor

from sklearn.ensemble import BaggingRegressor, GradientBoostingRegressor


from sklearn.neighbors import KNeighborsRegressor

        
# Dataset splitting        
from sklearn.model_selection import train_test_split


from matplotlib import pyplot as plt


import os
import joblib

In [2]:
df = pd.read_csv("dataset_scaled.csv")
df

,Unnamed: 0,in_amount_mmol,p_amount_mmol,sol_amount_ml,TOP_amount_mmol,acid_amount_mmol,amine_amount_mmol,thiol_amount_mmol,zinc_amount_mmol,other_amount_mmol,...,x8_copper bromide,x8_superhydride,x8_tetrabutylammonium myristate,x8_trioctylamine,x8_trioctylphosphine oxide,x8_water,x8_zinc iodide,diameter_nm,abs_nm,emission_nm
0,0,-0.205407,-0.376673,1.638502,-0.220165,0.095362,-0.800754,0.0,-0.682991,2.533209,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,3.5,562,618
1,1,-0.205407,-0.376673,1.638502,-0.220165,0.095362,-0.800754,0.0,-0.682991,2.533209,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,None,473,None
2,2,-0.205407,-0.376673,1.638502,-0.220165,0.095362,-0.800754,0.0,-0.682991,2.533209,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,None,524,None
3,3,-0.205407,-0.376673,1.638502,-0.220165,0.095362,-0.800754,0.0,-0.682991,2.533209,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,None,538,None
4,4,-0.205407,-0.376673,1.638502,-0.220165,0.095362,-0.800754,0.0,-0.682991,2.533209,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,None,576,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152,152,-0.467859,-0.729457,-0.258929,0.421871,-0.453863,-0.800754,0.0,-0.682991,-0.239089,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,None,552,601
153,153,-0.717189,-0.779855,-0.069186,-0.220165,-0.344018,-0.800754,0.0,-0.682991,-0.239089,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.3,532,590
154,154,-0.717189,-0.779855,-0.069186,-0.220165,-0.344018,-0.800754,0.0,-0.682991,-0.239089,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,None,570,None
155,155,-0.717189,-0.779855,-0.069186,-0.220165,-0.344018,-0.800754,0.0,-0.682991,-0.239089,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,None,562,None


In [5]:
# Input for ML models

input_col = ['in_amount_mmol',
             'p_amount_mmol',
             'sol_amount_ml',
             'TOP_amount_mmol',
             'acid_amount_mmol',
             'amine_amount_mmol',
             'thiol_amount_mmol',
             'zinc_amount_mmol',
             'other_amount_mmol',
             'total_volume_ml',
             'temp_c',
             'time_min',
             'x0_indium acetate',
             'x0_indium bromide',
             'x0_indium chloride',
             'x0_indium iodide',
             'x0_indium myristate',
             'x0_indium trifluoroacetate',
             'x1_bis(trimethylsilyl)phosphine',
             'x1_phosphorus trichloride',
             'x1_tris(diethylamino)phosphine',
             'x1_tris(dimethylamino)phosphine',
             'x1_tris(trimethylgermyl)phosphine',
             'x1_tris(trimethylsilyl)phosphine',
             'x2_None',
             'x2_octadecene',
             'x2_toluene',
             'x3_None',
             'x3_trioctylphosphine',
             'x4_None',
             'x4_lauric acid',
             'x4_myristic acid',
             'x4_oleic acid',
             'x4_palmitic acid',
             'x4_stearic acid',
             'x5_None',
             'x5_dioctylamine',
             'x5_dodecylamine',
             'x5_hexadecylamine',
             'x5_octylamine',
             'x5_oleylamine',
             'x6_None',
             'x7_None',
             'x7_zinc bromide',
             'x7_zinc chloride',
             'x7_zinc iodide',
             'x7_zinc oleate',
             'x7_zinc stearate',
             'x7_zinc undecylenate',
             'x8_None',
             'x8_acetic acid',
             'x8_copper bromide',
             'x8_superhydride',
             'x8_tetrabutylammonium myristate',
             'x8_trioctylamine',
             'x8_trioctylphosphine oxide',
             'x8_water',
             'x8_zinc iodide',
            
            ]

In [6]:
# Load ML model for predicting absorbance

loaded_rf_ab = joblib.load('model_aug_abs_ExtraTrees.joblib')

# Replace 'None' entries in 'abs_nm' column by predicted values.

a = 0
for index, row in df.iterrows():
    if row['abs_nm'] == 'None':
        X = df.loc[index, input_col].to_numpy()
        df.loc[index, 'abs_nm'] = loaded_rf_ab.predict(X.reshape(1, -1))[0]
        a += 1

        
# Save the dataset where all 'None' values in 'Absorbane max (nm)' column are replaced.   
df.to_csv('dataset_augmented_abs.csv')
df_first_aug = pd.read_csv('dataset_augmented_abs.csv')

In [7]:
#Saves the row indexes to drop for absorbance modelling into a list
total_row_num = len(df_first_aug)
drop_list_emi =[]
for row_i in range(total_row_num):
    if df_first_aug['emission_nm'].values[row_i] == 'None':
        drop_list_emi.append(row_i)
    
len(drop_list_emi)

#Drops the appropriate rows
df_emission_scaled_encoded = df_first_aug.drop(drop_list_emi)
#Saves the data for absorbance modelling to CSV
df_emission_scaled_encoded.to_csv('dataset_scaled_emission.csv')

# Move to **'model hao emission'** notebook to optimize model for augmenting emission

In [8]:
# adding column 'abs_nm' to input
input_col.append('abs_nm')
input_col_2 = input_col.copy()

In [9]:
print(input_col_2)
input_col.remove('abs_nm')

['in_amount_mmol', 'p_amount_mmol', 'sol_amount_ml', 'TOP_amount_mmol', 'acid_amount_mmol', 'amine_amount_mmol', 'thiol_amount_mmol', 'zinc_amount_mmol', 'other_amount_mmol', 'total_volume_ml', 'temp_c', 'time_min', 'x0_indium acetate', 'x0_indium bromide', 'x0_indium chloride', 'x0_indium iodide', 'x0_indium myristate', 'x0_indium trifluoroacetate', 'x1_bis(trimethylsilyl)phosphine', 'x1_phosphorus trichloride', 'x1_tris(diethylamino)phosphine', 'x1_tris(dimethylamino)phosphine', 'x1_tris(trimethylgermyl)phosphine', 'x1_tris(trimethylsilyl)phosphine', 'x2_None', 'x2_octadecene', 'x2_toluene', 'x3_None', 'x3_trioctylphosphine', 'x4_None', 'x4_lauric acid', 'x4_myristic acid', 'x4_oleic acid', 'x4_palmitic acid', 'x4_stearic acid', 'x5_None', 'x5_dioctylamine', 'x5_dodecylamine', 'x5_hexadecylamine', 'x5_octylamine', 'x5_oleylamine', 'x6_None', 'x7_None', 'x7_zinc bromide', 'x7_zinc chloride', 'x7_zinc iodide', 'x7_zinc oleate', 'x7_zinc stearate', 'x7_zinc undecylenate', 'x8_None', 'x8

In [10]:
# Now, the dataset only has 'None' values in the 'emission_nm' column

df_emi = pd.read_csv('dataset_augmented_abs.csv') 

# Load ML model for predicting emission

loaded_rf_emi = joblib.load('model_aug_emission_ExtraTrees.joblib')


# # Replace 'None' entries in 'emission_nm' column by predicted values.

a = 0
for index, row in df_emi.iterrows():
    if row['emission_nm'] == 'None':
        X = df_emi.loc[index, input_col_2].to_numpy()
        df_emi.loc[index, 'emission_nm'] = loaded_rf_emi.predict(X.reshape(1, -1))[0]
        a += 1

# # # Save the dataset where all 'None' values are replaced.

# # # Ready to use for other analysis.

df_emi.to_csv('dataset_augmented_abs_emission.csv')


## Aldjust dataset_augmented_abs_emission.csv to dataset_augmented_abs_emission_adjusted.csv

In [11]:
# adding column 'emission_nm' to input
input_col_2.append('emission_nm')
input_col_3 = input_col_2.copy()


In [12]:
print(input_col_3)
input_col_2.remove('emission_nm')

['in_amount_mmol', 'p_amount_mmol', 'sol_amount_ml', 'TOP_amount_mmol', 'acid_amount_mmol', 'amine_amount_mmol', 'thiol_amount_mmol', 'zinc_amount_mmol', 'other_amount_mmol', 'total_volume_ml', 'temp_c', 'time_min', 'x0_indium acetate', 'x0_indium bromide', 'x0_indium chloride', 'x0_indium iodide', 'x0_indium myristate', 'x0_indium trifluoroacetate', 'x1_bis(trimethylsilyl)phosphine', 'x1_phosphorus trichloride', 'x1_tris(diethylamino)phosphine', 'x1_tris(dimethylamino)phosphine', 'x1_tris(trimethylgermyl)phosphine', 'x1_tris(trimethylsilyl)phosphine', 'x2_None', 'x2_octadecene', 'x2_toluene', 'x3_None', 'x3_trioctylphosphine', 'x4_None', 'x4_lauric acid', 'x4_myristic acid', 'x4_oleic acid', 'x4_palmitic acid', 'x4_stearic acid', 'x5_None', 'x5_dioctylamine', 'x5_dodecylamine', 'x5_hexadecylamine', 'x5_octylamine', 'x5_oleylamine', 'x6_None', 'x7_None', 'x7_zinc bromide', 'x7_zinc chloride', 'x7_zinc iodide', 'x7_zinc oleate', 'x7_zinc stearate', 'x7_zinc undecylenate', 'x8_None', 'x8

In [14]:
df_second_aug = pd.read_csv('dataset_augmented_abs_emission_adjusted.csv')
#Saves the row indexes to drop for diameter modelling into a list
total_row_num = len(df_second_aug)
drop_list_dia =[]
for row_i in range(total_row_num):
    if df_second_aug['diameter_nm'].values[row_i] == 'None':
        drop_list_dia.append(row_i)
    
len(drop_list_dia)

#Drops the appropriate rows
df_dia_scaled_encoded = df_second_aug.drop(drop_list_dia)
#Saves the data for absorbance modelling to CSV
df_dia_scaled_encoded.to_csv('dataset_scaled_diameter.csv')

# Move to **'model hao diameter'** notebook to optimize model for augmenting emission.


In [15]:
# Load ML model for predicting diameters

df_dia = pd.read_csv('dataset_augmented_abs_emission_adjusted.csv') 

loaded_rf_dia = joblib.load('model_aug_diameter_RandomForest.joblib')

# Replace 'None' entries in 'diameter_nm' column by predicted values.

a = 0
for index, row in df_dia.iterrows():
    if row['diameter_nm'] == 'None':
        X = df_dia.loc[index, input_col_3].to_numpy()
        df_dia.loc[index, 'diameter_nm'] = loaded_rf_dia.predict(X.reshape(1, -1))[0]
        a += 1

        
# Save the dataset where all 'None' values in 'diameter_nm' column are replaced.   
df_dia.to_csv('dataset_augmented.csv')